In [1]:
import glob
import pandas as pd
import json
import datetime
from tqdm import tqdm, tqdm_notebook
import os
from django.conf import settings
#from sqlalchemy import create_engine
os.environ["DJANGO_SETTINGS_MODULE"] = "mysite.settings"
import django
django.setup()
from tqdm import tqdm_notebook
import glob
import numpy as np
from django.db.models import Min,Max,Count
comfig_file_path = '../../main_config.txt'
main_name = 'yandex/'
sub_name = 'raw_org_search/'

/home/tserekh/env/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
import sys
sys.path.append('/usr/lib/python3/dist-packages')
from pyproj import Proj, transform

PROJ3857  = Proj("+init=EPSG:3857")
PROJ4326 = Proj("+init=EPSG:4326")

In [3]:
from main_map.models import AlcoStopper as OrganizationNatClass

In [4]:
def get_words_dates(path):
    filename= '.'.join(path.split('/')[-1].split('.')[:-1])
    date = filename[-17:]
    date = datetime.datetime.strptime(date, '%Y-%m-%d_%H%M%S')
    word = filename[:-17]
    return (word, date,path)

In [5]:
paths = glob.glob('/home/tserekh/abc_data/yandex/raw_org_search_json/*.json')
df_paths = pd.DataFrame(paths,columns = ['path'])

df_paths['word'] = df_paths['path'].apply(lambda path: get_words_dates(path)[0])
df_paths['date'] = df_paths['path'].apply(lambda path: get_words_dates(path)[1])
df_paths = df_paths[df_paths['date']>pd.datetime(2018,12,7,1,4)]
df_paths = df_paths.sort_values('date', ascending=False)
df_paths = df_paths.drop_duplicates('word',keep='first')

paths = df_paths['path'].values

In [6]:
paths

array(['/home/tserekh/abc_data/yandex/raw_org_search_json/Автомойка2019-05-25_003336.json',
       '/home/tserekh/abc_data/yandex/raw_org_search_json/Автоаксессуары2019-05-25_001918.json',
       '/home/tserekh/abc_data/yandex/raw_org_search_json/АЗС2019-05-25_000524.json',
       '/home/tserekh/abc_data/yandex/raw_org_search_json/Магазин продуктов2019-05-24_235129.json',
       '/home/tserekh/abc_data/yandex/raw_org_search_json/Супермаркет2019-05-24_233323.json',
       '/home/tserekh/abc_data/yandex/raw_org_search_json/Ресторан2019-05-24_231847.json',
       '/home/tserekh/abc_data/yandex/raw_org_search_json/Парикмахерская2019-05-24_230212.json',
       '/home/tserekh/abc_data/yandex/raw_org_search_json/Аптека2019-05-24_224425.json',
       '/home/tserekh/abc_data/yandex/raw_org_search_json/Банкомат2019-05-24_222832.json',
       '/home/tserekh/abc_data/yandex/raw_org_search_json/Банк2019-05-24_221318.json'],
      dtype=object)

In [7]:
#paths = ['/home/tserekh/abc_data/yandex/raw_org_search_json/Кафе2018-12-02_120706.json']

In [8]:
# OrganizationNatClass.objects.all().delete()

In [9]:
df_all = pd.DataFrame()
for path in tqdm_notebook(paths):
    
    ids = []
    names = []
    coords = []
    chain_names = []
    lats = []
    lons = []
    addresses = []
    urls = []
    
    print(path.split('/')[-1], end = ' ')
    
    f = open(path)
    line = f.readline()
    f.close()
    struct = json.loads(line)

    for j in range(len(struct)):
        
        if str(struct[j])=="{'status': 'error', 'message': 'internal server error', 'code': 500}":
            print('error')
            continue
        features = struct[j]['features']
        for feature in features:
            id_ = feature['properties']['CompanyMetaData']['id']
            chain_name = feature['properties']['CompanyMetaData']['name']
            lon, lat = feature['geometries'][0]['coordinates']
            address = feature['properties']['CompanyMetaData']['address']
            if 'url' in feature['properties']['CompanyMetaData'].keys():
                url = feature['properties']['CompanyMetaData']['url']
            else:
                url = None
            for el in feature['properties']['CompanyMetaData']['Categories']:
                names.append(el['name'])
                ids.append(id_)
                urls.append(url)
                chain_names.append(chain_name)
                lats.append(lat)
                lons.append(lon)
                addresses.append(address)

    df = pd.DataFrame([names,ids,chain_names, lats, lons,addresses,urls]).T
    features = ['nat_class', 'source_id','chain_name','lat','lon','address','source_name']
    df.columns = features
    df = df.drop_duplicates(['nat_class','source_id'])
    

    
    df['source_name'] = 'yandex_api'
    
    #df = df[df['nat_class'].apply(lambda x: x.lower()).isin(df_paths['word'].apply(lambda x: x.lower()).values)]
    print(len(df))
    df = df.drop_duplicates(['nat_class','source_id'])
    print(len(df))
    
    for i, row in tqdm_notebook(df[features].iterrows(),leave=False):
        org = OrganizationNatClass(**row.to_dict())
        org.x, org.y = transform(PROJ4326, PROJ3857, row['lon'], row['lat'])
        org.save()

Автомойка2019-05-25_003336.json 10793
10793


Автоаксессуары2019-05-25_001918.json 8977
8977


АЗС2019-05-25_000524.json 2648
2648


Магазин продуктов2019-05-24_235129.json 44914
44914


Супермаркет2019-05-24_233323.json 12467
12467


Ресторан2019-05-24_231847.json 17750
17750


Парикмахерская2019-05-24_230212.json 40764
40764


Аптека2019-05-24_224425.json 10420
10420


Банкомат2019-05-24_222832.json 13707
13707


Банк2019-05-24_221318.json 6468
6468


In [12]:
unique_fields = ['source_id', 'source_name','nat_class']

duplicates = (
    OrganizationNatClass.objects.values(*unique_fields)
    .order_by()
    .annotate(max_id=Max('id'), count_id=Count('id'))
    .filter(count_id__gt=1)
)
print(len(duplicates))

47065


In [13]:
for duplicate in tqdm_notebook(duplicates):
    (
        OrganizationNatClass.objects
        .filter(**{x: duplicate[x] for x in unique_fields})
        .exclude(id=duplicate['max_id'])
        .delete()
    )

In [ ]:
# print(Organization.objects.all().delete())
# print(OrganizationNatClass.objects.all().delete())


In [ ]:
model_list = [OrganizationNatClass]
for Model in model_list:
    for el in tqdm(Model.objects.filter(x__isnull=True)):
        el.x, el.y = transform(PROJ4326, PROJ3857, el.lon, el.lat)
        el.save()